### Task 0 (setup)

In [31]:
import streamlit as st
from streamlit_folium import st_folium

import geopandas as gpd
from geopandas import GeoSeries
from shapely.geometry import Point, LineString

import folium
from folium import Marker, GeoJson
from folium.plugins import MarkerCluster

import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import matplotlib.pyplot as plt 
import chardet



In [ ]:
base = open(r'data/hospitals.csv', 'rb').read()
det = chardet.detect(base)
charenc = det['encoding']
charenc

In [32]:
# --- Importamos shapefiles y csv ---
districts = gpd.read_file(r'data/shape_file/DISTRITOS.shp').to_crs("EPSG:4326")
hospitals = pd.read_csv("data/hospitals.csv", encoding = "latin1")
# Limpio los NaN
hospitals= hospitals.dropna(subset=["ESTE", "NORTE"])

hospitals.head( 5 )

,Institución,Código Único,Nombre del establecimiento,Clasificación,Tipo,Departamento,Provincia,Distrito,UBIGEO,Dirección,...,Inicio de Actividad,Director Médico y/o Responsable de la Atención de Salud,Estado,Situación,Condición,Inspección,NORTE,ESTE,COTA,CAMAS
1,GOBIERNO REGIONAL,7050,AMBATO,PUESTOS DE SALUD O POSTAS DE SALUD,ESTABLECIMIENTO DE SALUD SIN INTERNAMIENTO,CAJAMARCA,CUTERVO,SANTA CRUZ,60611,COMUNIDAD DE AMBATO,...,29/06/2006,IDELSO MENOR CHAVEZ,ACTIVADO,,EN FUNCIONAMIENTO,,-78.858380,-6.133523,1724.0,NaN
2,GOBIERNO REGIONAL,99,SANTA ISABEL DE YUMBATURO,PUESTOS DE SALUD O POSTAS DE SALUD,ESTABLECIMIENTO DE SALUD SIN INTERNAMIENTO,LORETO,LORETO,PARINARI,160302,ACTUALIZAR,...,01/01/1900,JULIO TONY CAITIMARI MACHOA,ACTIVADO,,EN FUNCIONAMIENTO,,-74.258139,-4.581509,124.0,NaN
6,MINSA,7278,PUESTO DE SALUD HEROES DEL CENEPA,PUESTOS DE SALUD O POSTAS DE SALUD,ESTABLECIMIENTO DE SALUD SIN INTERNAMIENTO,LIMA,LIMA,VILLA EL SALVADOR,150142,"JR. HEROES DEL CENEPA MZ C LT 20, ALT DE PANAM...",...,01/02/2008,BASTIDAS CAMARENA HUBERT ALEX . .,ACTIVADO,,EN FUNCIONAMIENTO,,-76.930608,-12.248699,NaN,NaN
12,GOBIERNO REGIONAL,5460,NUEVA BETANIA,PUESTOS DE SALUD O POSTAS DE SALUD,ESTABLECIMIENTO DE SALUD SIN INTERNAMIENTO,UCAYALI,CORONEL PORTILLO,CALLERIA,250101,OTROS CC.NN. NUEVA BETANIA S/N NÚMERO S/N DIST...,...,20/12/2000,MARIANA VASQUEZ PINEDO,ACTIVADO,,EN FUNCIONAMIENTO,,-74.296531,-8.398366,177.0,NaN
15,GOBIERNO REGIONAL,6431,PONGO ISLA,PUESTOS DE SALUD O POSTAS DE SALUD,ESTABLECIMIENTO DE SALUD SIN INTERNAMIENTO,SAN MARTIN,SAN MARTIN,HUIMBAYOC,220907,OTROS CP PONGO ISLA DISTRITO HUIMBAYOC PROVINC...,...,14/09/2001,MIRLANDA MOZOMBITE BARDALES,ACTIVADO,,EN FUNCIONAMIENTO,,-75.885812,-6.438298,172.1,NaN


### Task 1

In [34]:
# Filtramos los hospitales públicos y en funcionamiento

hospitals_public = hospitals[
    (hospitals["Estado"].str.upper() == "ACTIVADO") &
    (hospitals["Condición"].str.upper() == "EN FUNCIONAMIENTO") &
    (hospitals["Institución"].str.contains("GOBIERNO|MINSA", case=False, na=False))
].copy()

# Creamos el GeoDataFrame

hospitals_public["geometry"] = hospitals_public.apply(
    lambda row: Point(float(row["ESTE"]), float(row["NORTE"])) 
    if pd.notna(row["ESTE"]) and pd.notna(row["NORTE"]) else None,
    axis=1
)

hospitals_gdf = gpd.GeoDataFrame(hospitals_public, geometry="geometry", crs="EPSG:4326")


# Contar hospitales por UBIGEO
hospital_count = hospitals_gdf.groupby("UBIGEO").size().reset_index(name="hospitales")

# Asegurar que UBIGEO esté como string para merge
districts["UBIGEO"] = districts["IDDIST"].astype(str).str.zfill(6)
hospital_count["UBIGEO"] = hospital_count["UBIGEO"].astype(str).str.zfill(6)

# Merge distritos con el conteo
districts_hosp = districts.merge(hospital_count, on="UBIGEO", how="left").fillna({"hospitales": 0})


# --- Mapa 1: Total hospitales ---
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
districts_hosp.plot(column="hospitales", cmap="Reds", legend=True, linewidth=0.2, edgecolor="black", ax=ax)
plt.title("Mapa 1: Total hospitales públicos por distrito")
plt.axis("off")
plt.savefig("output/mapa1.png", dpi=300, bbox_inches="tight")
plt.close()


# --- Mapa 2: Distritos con cero hospitales ---
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
districts_hosp.plot(color="lightgrey", edgecolor="black", linewidth=0.2, ax=ax)
districts_hosp[districts_hosp["hospitales"] == 0].plot(color="red", ax=ax, label="0 hospitales")
plt.title("Mapa 2: Distritos con cero hospitales")
plt.savefig("output/mapa2.png", dpi=300, bbox_inches="tight")
plt.close()

# --- Mapa 3: Top 10 distritos con más hospitales ---
top10 = districts_hosp.nlargest(10, "hospitales")

fig, ax = plt.subplots(1, 1, figsize=(10, 10))
districts_hosp.plot(color="lightgrey", edgecolor="black", linewidth=0.2, ax=ax)
top10.plot(column="hospitales", cmap="OrRd", legend=True, edgecolor="black", linewidth=0.5, ax=ax)
plt.title("Mapa 3: Top 10 distritos con más hospitales")
plt.axis("off")
plt.savefig("output/mapa3.png", dpi=300, bbox_inches="tight")
plt.close()

### Task 2

In [36]:

# Conteo por departamento 
dept_hosp = hospitals.groupby("Departamento").size().reset_index(name="hospitales")

# Ordenar de mayor a menor
dept_hosp = dept_hosp.sort_values("hospitales", ascending=False).reset_index(drop=True)

print("Resumen de hospitales por departamento:")
print(dept_hosp)

# Identificar máximo y mínimo
max_dept = dept_hosp.iloc[0]
min_dept = dept_hosp.iloc[-1]

print(f"\nDepto con MÁS hospitales: {max_dept['Departamento']} ({max_dept['hospitales']})")
print(f"Depto con MENOS hospitales: {min_dept['Departamento']} ({min_dept['hospitales']})")

# Bar chart
plt.figure(figsize=(12, 6))
plt.bar(dept_hosp["Departamento"], dept_hosp["hospitales"], color="skyblue")
plt.xticks(rotation=90)
plt.title("Número de hospitales por departamento")
plt.xlabel("Departamento")
plt.ylabel("Hospitales")
plt.tight_layout()
plt.savefig("output/mapa4.png", dpi=300, bbox_inches="tight")
plt.close()

# Agrupamos el shapefile a nivel de departamento
departments = districts.dissolve(by="DEPARTAMEN").reset_index()

# Merge con conteo de hospitales
departments = departments.merge(dept_hosp, left_on="DEPARTAMEN", right_on="Departamento", how="left")
departments["hospitales"] = departments["hospitales"].fillna(0)

# Plot choropleth
fig, ax = plt.subplots(1, 1, figsize=(10, 8))
departments.plot(
    column="hospitales",
    cmap="YlGnBu",
    legend=True,
    edgecolor="black",
    linewidth=0.4,
    ax=ax,
    legend_kwds={"label": "Número de hospitales", "orientation": "vertical"}
)
plt.title("Mapa de hospitales por departamento", fontsize=14, fontweight="bold")
plt.axis("off")
plt.savefig("output/mapa5.png", dpi=300, bbox_inches="tight")
plt.close()

Resumen de hospitales por departamento:
     Departamento  hospitales
0       CAJAMARCA         849
1            LIMA         776
2           JUNIN         493
3           PIURA         444
4          ANCASH         418
5        AMAZONAS         417
6            PUNO         407
7        AYACUCHO         387
8      SAN MARTIN         379
9        APURIMAC         375
10         LORETO         353
11          CUSCO         339
12    LA LIBERTAD         336
13   HUANCAVELICA         325
14        HUANUCO         261
15          PASCO         258
16        UCAYALI         234
17       AREQUIPA         224
18     LAMBAYEQUE         183
19            ICA         146
20         CALLAO          97
21          TACNA          79
22  MADRE DE DIOS          66
23       MOQUEGUA          56
24         TUMBES          54

Depto con MÁS hospitales: CAJAMARCA (849)
Depto con MENOS hospitales: TUMBES (54)


### Task 3

In [38]:
# Paths
BASE = Path = __import__("pathlib").Path
BASE_DIR = Path.cwd()
SHAPE_DIR = BASE_DIR / "data" / "shape_file"
POP_FILE = SHAPE_DIR / "CCPP_IGN100K.shp"
DISTRICTS_FILE = SHAPE_DIR / "DISTRITOS.shp"
HOSP_CSV = BASE_DIR / "data" / "hospitals.csv" 

# CRS
CRS_WGS84 = "EPSG:4326"
CRS_UTM18S = "EPSG:32718"   

# --- Funciones ---
def load_hospitals_csv(csv_path):
    """Carga hospitales desde CSV y crea GeoDataFrame de puntos válidos"""
    df = pd.read_csv(csv_path, encoding="latin1")
    df.columns = [c.strip() for c in df.columns]

    lon = next((c for c in df.columns if c.upper() in ["ESTE","LONG","LON","X"]), None)
    lat = next((c for c in df.columns if c.upper() in ["NORTE","LAT","Y"]), None)
    df = df.dropna(subset=[lon, lat])

    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df[lon], df[lat]), crs=CRS_WGS84)
    return gdf[gdf.geometry.is_valid].drop_duplicates(subset=["geometry"])

def load_popcenters(shp_path):
    """Carga centros poblados y usa centroides si no son puntos"""
    gdf = gpd.read_file(shp_path)
    gdf = gdf.set_crs(CRS_WGS84) if gdf.crs is None else gdf.to_crs(CRS_WGS84)

    if not set(gdf.geometry.geom_type.unique()) <= {"Point"}:
        gdf["geometry"] = gdf.geometry.centroid

    gdf.columns = [c.strip() for c in gdf.columns]
    return gdf

def proximity_for_region(pop_gdf, hosp_gdf, region_name, buffer_m=10000):
    """Cuenta hospitales dentro de buffer (10 km) por centro poblado en una región"""
    # Columna de departamento
    region_col = next((c for c in pop_gdf.columns if c.upper() in ["DEP","DEPARTAMENTO","DEPTO"]), None)

    pop_sel = pop_gdf[pop_gdf[region_col].str.contains(region_name, case=False, na=False)].copy()
    pop_m = pop_sel.to_crs(CRS_UTM18S)
    hosp_m = hosp_gdf.to_crs(CRS_UTM18S)
    sidx = hosp_m.sindex

    counts, buffers, hosp_ids = [], [], []
    for _, row in pop_m.iterrows():
        buf = row.geometry.buffer(buffer_m)
        buffers.append(buf)
        possible = hosp_m.iloc[list(sidx.intersection(buf.bounds))]
        inside = possible[possible.geometry.within(buf)]
        counts.append(len(inside))
        hosp_ids.append(list(inside.index))

    pop_sel["hosp_in_10km"] = counts
    pop_sel["buffer_geom"] = gpd.GeoSeries(buffers, crs=CRS_UTM18S).to_crs(CRS_WGS84).values
    pop_sel["hosp_ids"] = hosp_ids
    pop_sel = pop_sel.to_crs(CRS_WGS84)

    return {
        "pop_sel": pop_sel,
        "min": pop_sel.loc[pop_sel["hosp_in_10km"].idxmin()],
        "max": pop_sel.loc[pop_sel["hosp_in_10km"].idxmax()]
    }

def build_folium_map(center_row, hosp_gdf, out_html, circle_color="red"):
    """Crea mapa Folium con centro poblado, buffer y hospitales cercanos"""
    m = folium.Map(location=[center_row.geometry.y, center_row.geometry.x], zoom_start=11)
    folium.GeoJson(center_row["buffer_geom"].__geo_interface__,
                   style_function=lambda feat: {"color": circle_color,"fill": False,"weight": 2}).add_to(m)
    folium.Marker([center_row.geometry.y, center_row.geometry.x],
                  popup=f"{center_row.get('NOM_POBLAD','Centro')} - {int(center_row['hosp_in_10km'])} hospitales").add_to(m)

    mc = MarkerCluster()
    for hid in center_row.get("hosp_ids", []):
        h = hosp_gdf.loc[hid]
        folium.Marker([h.geometry.y, h.geometry.x],
                      popup=h.get("Nombre del establecimiento", "Hospital")).add_to(mc)
    mc.add_to(m)

    m.save(out_html)
    return m

# --- Ejecución principal solo para Lima y Loreto ---
def run_proximity_tasks():
    hospitals_gdf = load_hospitals_csv(HOSP_CSV)
    pop_gdf = load_popcenters(POP_FILE)
    hospitals_gdf = hospitals_gdf.drop_duplicates(subset=["geometry"])
    pop_gdf = pop_gdf[pop_gdf.geometry.is_valid].copy()

    regions = ["LIMA", "LORETO"]  # Solo estas dos regiones
    out_dir = BASE_DIR / "output"
    out_dir.mkdir(exist_ok=True)
    outputs = {}

    for region in regions:
        res = proximity_for_region(pop_gdf, hospitals_gdf, region)
        pop_sel, min_row, max_row = res["pop_sel"], res["min"], res["max"]

        pop_sel[["NOM_POBLAD","hosp_in_10km"]].to_csv(out_dir / f"proximity_summary_{region}.csv", index=False)

        build_folium_map(min_row, hospitals_gdf, out_dir / f"proximity_{region}_min.html", "red")
        build_folium_map(max_row, hospitals_gdf, out_dir / f"proximity_{region}_max.html", "green")

        outputs[region] = {"summary_gdf": pop_sel, "min": min_row, "max": max_row}

    return outputs

if __name__ == "__main__":
    outputs = run_proximity_tasks()
    print("Proceso completado.")

Proceso completado.


### Task 4

In [39]:
# --- Asegurar que UBIGEO sea string con 6 dígitos ---
districts["UBIGEO"] = districts["UBIGEO"].astype(str).str.zfill(6)
hospitals["UBIGEO"] = hospitals["UBIGEO"].astype(str).str.zfill(6)

# --- Conteo de hospitales por distrito ---
hospital_count = hospitals.groupby("UBIGEO").size().reset_index(name="Hosp_Count")
districts = districts.merge(hospital_count, on="UBIGEO", how="left")
districts["Hosp_Count"] = districts["Hosp_Count"].fillna(0)

# --- Crear mapa base ---
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=[-9.19, -75.0152], zoom_start=5, tiles="cartodbpositron")

# --- Choropleth ---
folium.Choropleth(
    geo_data=districts,
    data=districts,
    columns=["UBIGEO", "Hosp_Count"],
    key_on="feature.properties.UBIGEO",
    fill_color="YlOrRd",
    fill_opacity=0.7,
    line_opacity=0.2,
    nan_fill_color="white",
    legend_name="Número de hospitales por distrito",
).add_to(m)

# --- Cluster de hospitales ---
marker_cluster = MarkerCluster().add_to(m)
for _, row in hospitals.iterrows():
    if pd.notnull(row["NORTE"]) and pd.notnull(row["ESTE"]):
        folium.Marker(
            location=[row["ESTE"], row["NORTE"]],
            popup=f"{row['Nombre del establecimiento']} ({row['Distrito']})"
        ).add_to(marker_cluster)

m.save("output/mapa_hospitales_distritos.html")

In [29]:
#Ahora sí importamos los centros poblados

popcenters = gpd.read_file(r'data/shape_file/CCPP_IGN100K.shp').to_crs("EPSG:4326")

### Task 5

In [ ]:
tooltip = "Click"

# --- Seleccionar departamentos Lima y Loreto ---
pop_lima = popcenters[popcenters["DEP"] == "LIMA"].to_crs(32718)
pop_loreto = popcenters[popcenters["DEP"] == "LORETO"].to_crs(32718)

hosp_proj = gpd.GeoDataFrame(
    hospitals,
    geometry=gpd.points_from_xy(hospitals["ESTE"], hospitals["NORTE"]),
    crs="EPSG:4326"
).to_crs(32718)

# Función para contar hospitales en 10 km
def contar_hospitales(poblado, hospitales):
    buffer = poblado.geometry.buffer(10000)
    return hospitales[hospitales.intersects(buffer)].shape[0]

# Calcular hospitales cercanos
pop_lima["hospitales_10km"] = pop_lima.apply(lambda row: contar_hospitales(row, hosp_proj), axis=1)
pop_loreto["hospitales_10km"] = pop_loreto.apply(lambda row: contar_hospitales(row, hosp_proj), axis=1)

# Selección de extremos (filas completas)
lima_min = pop_lima.loc[pop_lima["hospitales_10km"].idxmin()]
lima_max = pop_lima.loc[pop_lima["hospitales_10km"].idxmax()]

loreto_min = pop_loreto.loc[pop_loreto["hospitales_10km"].idxmin()]
loreto_max = pop_loreto.loc[pop_loreto["hospitales_10km"].idxmax()]

# Reproyectar a WGS84 para Folium
lima_min_geom = gpd.GeoSeries([lima_min.geometry], crs=32718).to_crs(4326).iloc[0]
lima_max_geom = gpd.GeoSeries([lima_max.geometry], crs=32718).to_crs(4326).iloc[0]
loreto_min_geom = gpd.GeoSeries([loreto_min.geometry], crs=32718).to_crs(4326).iloc[0]
loreto_max_geom = gpd.GeoSeries([loreto_max.geometry], crs=32718).to_crs(4326).iloc[0]

# --- MAPA ---
m = folium.Map(location=[-9, -75], zoom_start=5)

#  Lima - menos hospitales
folium.Circle(
    [lima_min_geom.y, lima_min_geom.x],
    popup=f"{lima_min['NOM_POBLAD']} - {lima_min['hospitales_10km']} hospitales",
    radius=10000,
    fill=True,
    color="red",
    tooltip=tooltip
).add_to(m)

#  Lima - más hospitales
folium.Circle(
    [lima_max_geom.y, lima_max_geom.x],
    popup=f"{lima_max['NOM_POBLAD']} - {lima_max['hospitales_10km']} hospitales",
    radius=10000,
    fill=True,
    color="green",
    tooltip=tooltip
).add_to(m)

#  Loreto - menos hospitales
folium.Circle(
    [loreto_min_geom.y, loreto_min_geom.x],
    popup=f"{loreto_min['NOM_POBLAD']} - {loreto_min['hospitales_10km']} hospitales",
    radius=10000,
    fill=True,
    color="red",
    tooltip=tooltip
).add_to(m)

#  Loreto - más hospitales
folium.Circle(
    [loreto_max_geom.y, loreto_max_geom.x],
    popup=f"{loreto_max['NOM_POBLAD']} - {loreto_max['hospitales_10km']} hospitales",
    radius=10000,
    fill=True,
    color="green",
    tooltip=tooltip
).add_to(m)

m.save("output/proximity_combinado.html")